In [4]:
# import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
# load the dataset
df = pd.read_csv('high_school_career_recommendation_dataset.csv')
df.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Split data into features and target
X = df.drop('Recommended_Career', axis=1)
y = df['Recommended_Career']

# One-hot encoding for categorical features
X = pd.get_dummies(X, columns=['Education', 'Interest', 'Favorite_Subject',
                       'Extracurriculars', 'Personality_Trait'])

# Splitting into train and test
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

#Second split to get validation and test data
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Scaling features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

y_train = tf.keras.utils.to_categorical(y_train_encoded, num_classes=len(label_encoder.classes_))
y_val = tf.keras.utils.to_categorical(y_val_encoded, num_classes=len(label_encoder.classes_))
y_test = tf.keras.utils.to_categorical(y_test_encoded, num_classes=len(label_encoder.classes_))

print('Training data shape', x_train_scaled.shape)
print('Validation data shape', x_val_scaled.shape)
print('Test data shape', x_test_scaled.shape)

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
import joblib

logistic_model = LogisticRegression(
    multi_class='multinomial', #for multiclass classification
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)

logistic_model.fit(x_train_scaled, y_train_encoded)
joblib.dump(logistic_model, 'saved_models/logistic_regression_model.pkl')

y_pred_train = logistic_model.predict(x_train_scaled)
y_pred_test = logistic_model.predict(x_test_scaled)

#Evaluate model
print(classification_report(y_train_encoded, y_pred_train, target_names=label_encoder.classes_))
print(classification_report(y_test_encoded, y_pred_test, target_names=label_encoder.classes_))

In [ ]:
def make_predictions(model_path, X):

    # Load the model
    model = joblib.load(model_path)
    # Make predictions
    predictions = model.predict(X)
    # Convert probabilities to binary labels (0 or 1)

    return predictions


In [ ]:
model_path = 'models/logistic_regression_model.pkl'
make_predictions(model_path, x_test_scaled)